In [133]:
import pymongo
import re
import nltk

def load_data(database_name, collection_name):
    # Connect to MongoDB
    client = pymongo.MongoClient("mongodb://localhost:27017/")
    db = client[database_name]
    collection = db[collection_name]

    # Load data from MongoDB
    data = list(collection.find())

    return data


 # Load data from MongoDB
database_name = "lab1"
collection_name = "diseases"
raw_data = load_data(database_name, collection_name)

In [62]:
raw_data[0]

{'_id': ObjectId('66137a12432dd74b0b4d7428'),
 'count': 1,
 'illness_type': 'داء السكري',
 'disease_name': 'التغذية الصحية لمرضى السكري المصابين بأمراض الكلى',
 'content': '<div id="ctl00_PlaceHolderMain_ctl02__ControlWrapper_RichHtmlField" class="ms-rtestate-field" style="display:inline" aria-labelledby="ctl00_PlaceHolderMain_ctl02_label"><p style="text-align:justify;">\u200b<strong>الخطة الغذائية لمرضى الكلى:</strong></p><p style="text-align:justify;">المكملات العشبية ليست آمنة في حالة مرضى الكلى، كما أن بعض الفيتامينات يمكن أن تضر مرضى الكلى أيضًا، لذلك يجب استشارة الطبيب قبل تناول أي نوع منها، وتعتمد الخطة الغذائية لمريض الكلى على حالة المريض وما إذا كان في مرحلة متقدمة أو متأخرة من المرض؛ وعليها يُوصَى بالحد من، أو تجنُّب بعض الأطعمة الغذائية، ويوصى لمريض الكلى بالآتي:<br></p><ul style="text-align:justify;"><li>الحد من الصوديوم: حيث مع مُضِي الوقت يفقد مريض الكلى تدريجيًّا القدرة على تحقيق التوازن بين المياه والصوديوم بالجسم؛ لذلك يساعد الحد من الصوديوم على خفض ضغط الدم وتقليل احت

In [154]:
# I changed the '' into ' ' to assure that no words get accidentally stuck together.
# removed the \u200b one
# found three articles with a different structure
# encoding was done throughly by using write for json, and then to mongoDB, using compass to visualize
# got the scraping by layers so that I would make sure to get good results
# made it exclude all the punctuations manually except for the shadda

import re

def clean_data(data):
    cleaned_data = []
    for entry in data:
        if 'content' in entry and entry['content']:
            # Remove HTML tags
            content = re.sub(r'<[^>]+>|[\u200b]', ' ', entry['content'])
            
            # Remove extra whitespaces
            content = re.sub(r'\s+', ' ', content).strip()

            # Remove parentheses
            content = content.replace('(', ' ').replace(')', ' ')
            
            # Add cleaned content to the entry
            cleaned_entry = {
                'illness_type': entry['illness_type'],
                'disease_name': entry['disease_name'],
                'content': content
            }

            cleaned_data.append(cleaned_entry)
        else:
            print("Skipping entry with missing or empty content:", entry)

    return cleaned_data

# Example usage
cleaned_data = clean_data(raw_data)


In [155]:
cleaned_data[0]

{'illness_type': 'داء السكري',
 'disease_name': 'التغذية الصحية لمرضى السكري المصابين بأمراض الكلى',
 'content': 'الخطة الغذائية لمرضى الكلى: المكملات العشبية ليست آمنة في حالة مرضى الكلى، كما أن بعض الفيتامينات يمكن أن تضر مرضى الكلى أيضًا، لذلك يجب استشارة الطبيب قبل تناول أي نوع منها، وتعتمد الخطة الغذائية لمريض الكلى على حالة المريض وما إذا كان في مرحلة متقدمة أو متأخرة من المرض؛ وعليها يُوصَى بالحد من، أو تجنُّب بعض الأطعمة الغذائية، ويوصى لمريض الكلى بالآتي: الحد من الصوديوم: حيث مع مُضِي الوقت يفقد مريض الكلى تدريجيًّا القدرة على تحقيق التوازن بين المياه والصوديوم بالجسم؛ لذلك يساعد الحد من الصوديوم على خفض ضغط الدم وتقليل احتباس السوائل بالجسم وهو أمر شائع مع مرضى الكلى. التركيز على الطعام الطازج والمطهو بالمنزل، وتناول كميات صغيرة من طعام المطاعم والأطعمة المعلبة لأنها غالبًا ما تحتوي على الكثير من الصوديوم. اختيار منتجات الغذاء التي تحتوي على نسبة صوديوم أقل من 5% على ملصقها الغذائي. استبدال الملح بتعزيز النكهات بالأعشاب والتوابل والخردل والخل وفي غضون أسبوع أو أسبوعين سيعتاد

In [ ]:
nltk.download('punkt')
from nltk.tokenize import word_tokenize

In [160]:
def tokenize_arabic_sentences(data):
    tokenized_data = []
    for entry in data:
        content = entry.get('content')
        if content:
            # Define a custom regex pattern to split Arabic text into sentences
            sentences = re.split(r'(?<=[.!?،])\s+(?=[^\sء])', content)
            entry_without_content = {key: value for key, value in entry.items() if key != 'content'}
            entry_without_content['sentences'] = sentences
            tokenized_data.append(entry_without_content)
    return tokenized_data

# Example usage:
tokenized_sentences_data = tokenize_arabic_sentences(cleaned_data)
tokenized_sentences_data[0]

{'illness_type': 'داء السكري',
 'disease_name': 'التغذية الصحية لمرضى السكري المصابين بأمراض الكلى',
 'sentences': ['الخطة الغذائية لمرضى الكلى: المكملات العشبية ليست آمنة في حالة مرضى الكلى،',
  'كما أن بعض الفيتامينات يمكن أن تضر مرضى الكلى أيضًا،',
  'لذلك يجب استشارة الطبيب قبل تناول أي نوع منها،',
  'وتعتمد الخطة الغذائية لمريض الكلى على حالة المريض وما إذا كان في مرحلة متقدمة أو متأخرة من المرض؛ وعليها يُوصَى بالحد من،',
  'أو تجنُّب بعض الأطعمة الغذائية،',
  'ويوصى لمريض الكلى بالآتي: الحد من الصوديوم: حيث مع مُضِي الوقت يفقد مريض الكلى تدريجيًّا القدرة على تحقيق التوازن بين المياه والصوديوم بالجسم؛ لذلك يساعد الحد من الصوديوم على خفض ضغط الدم وتقليل احتباس السوائل بالجسم وهو أمر شائع مع مرضى الكلى.',
  'التركيز على الطعام الطازج والمطهو بالمنزل،',
  'وتناول كميات صغيرة من طعام المطاعم والأطعمة المعلبة لأنها غالبًا ما تحتوي على الكثير من الصوديوم.',
  'اختيار منتجات الغذاء التي تحتوي على نسبة صوديوم أقل من 5% على ملصقها الغذائي.',
  'استبدال الملح بتعزيز النكهات بالأعشاب والتواب

In [135]:
#Word Tokenizing
def tokenize_sentences(data):
    tokenized_data = []
    for entry in data:
        content = entry.get('content')
        if content:
            words = word_tokenize(content)
            entry_without_content = {key: value for key, value in entry.items() if key != 'content'}
            entry_without_content['words'] = words
            tokenized_data.append(entry_without_content)
    return tokenized_data

# Example usage:
tokenized_data = tokenize_sentences(cleaned_data)
tokenized_data[0]

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


{'illness_type': 'داء السكري',
 'disease_name': 'التغذية الصحية لمرضى السكري المصابين بأمراض الكلى',
 'words': ['الخطة',
  'الغذائية',
  'لمرضى',
  'الكلى',
  'المكملات',
  'العشبية',
  'ليست',
  'آمنة',
  'في',
  'حالة',
  'مرضى',
  'الكلى',
  'كما',
  'أن',
  'بعض',
  'الفيتامينات',
  'يمكن',
  'أن',
  'تضر',
  'مرضى',
  'الكلى',
  'أيضًا',
  'لذلك',
  'يجب',
  'استشارة',
  'الطبيب',
  'قبل',
  'تناول',
  'أي',
  'نوع',
  'منها',
  'وتعتمد',
  'الخطة',
  'الغذائية',
  'لمريض',
  'الكلى',
  'على',
  'حالة',
  'المريض',
  'وما',
  'إذا',
  'كان',
  'في',
  'مرحلة',
  'متقدمة',
  'أو',
  'متأخرة',
  'من',
  'المرض',
  'وعليها',
  'يُوصَى',
  'بالحد',
  'من',
  'أو',
  'تجنُّب',
  'بعض',
  'الأطعمة',
  'الغذائية',
  'ويوصى',
  'لمريض',
  'الكلى',
  'بالآتي',
  'الحد',
  'من',
  'الصوديوم',
  'حيث',
  'مع',
  'مُضِي',
  'الوقت',
  'يفقد',
  'مريض',
  'الكلى',
  'تدريجيًّا',
  'القدرة',
  'على',
  'تحقيق',
  'التوازن',
  'بين',
  'المياه',
  'والصوديوم',
  'بالجسم',
  'لذلك',
  'يساعد',
  

In [136]:
nltk.download('stopwords')
from nltk.corpus import stopwords

print(stopwords.words('arabic'))

['إذ', 'إذا', 'إذما', 'إذن', 'أف', 'أقل', 'أكثر', 'ألا', 'إلا', 'التي', 'الذي', 'الذين', 'اللاتي', 'اللائي', 'اللتان', 'اللتيا', 'اللتين', 'اللذان', 'اللذين', 'اللواتي', 'إلى', 'إليك', 'إليكم', 'إليكما', 'إليكن', 'أم', 'أما', 'أما', 'إما', 'أن', 'إن', 'إنا', 'أنا', 'أنت', 'أنتم', 'أنتما', 'أنتن', 'إنما', 'إنه', 'أنى', 'أنى', 'آه', 'آها', 'أو', 'أولاء', 'أولئك', 'أوه', 'آي', 'أي', 'أيها', 'إي', 'أين', 'أين', 'أينما', 'إيه', 'بخ', 'بس', 'بعد', 'بعض', 'بك', 'بكم', 'بكم', 'بكما', 'بكن', 'بل', 'بلى', 'بما', 'بماذا', 'بمن', 'بنا', 'به', 'بها', 'بهم', 'بهما', 'بهن', 'بي', 'بين', 'بيد', 'تلك', 'تلكم', 'تلكما', 'ته', 'تي', 'تين', 'تينك', 'ثم', 'ثمة', 'حاشا', 'حبذا', 'حتى', 'حيث', 'حيثما', 'حين', 'خلا', 'دون', 'ذا', 'ذات', 'ذاك', 'ذان', 'ذانك', 'ذلك', 'ذلكم', 'ذلكما', 'ذلكن', 'ذه', 'ذو', 'ذوا', 'ذواتا', 'ذواتي', 'ذي', 'ذين', 'ذينك', 'ريث', 'سوف', 'سوى', 'شتان', 'عدا', 'عسى', 'عل', 'على', 'عليك', 'عليه', 'عما', 'عن', 'عند', 'غير', 'فإذا', 'فإن', 'فلا', 'فمن', 'في', 'فيم', 'فيما', 'فيه', 'فيها', '

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [137]:
def apply_stopwords(tokenized_data):
    stop_words = set(stopwords.words('arabic'))  # Arabic stopwords
    stopwords_data = []
    for entry in tokenized_data:
        words_without_stopwords = [word for word in entry['words'] if word not in stop_words]
        entry_without_stopwords = {
            'illness_type': entry['illness_type'],
            'disease_name': entry['disease_name'],
            'words': words_without_stopwords
        }
        stopwords_data.append(entry_without_stopwords)
    return stopwords_data

# Example usage:
stopwords_data = apply_stopwords(tokenized_data)
stopwords_data[0]


{'illness_type': 'داء السكري',
 'disease_name': 'التغذية الصحية لمرضى السكري المصابين بأمراض الكلى',
 'words': ['الخطة',
  'الغذائية',
  'لمرضى',
  'الكلى',
  'المكملات',
  'العشبية',
  'آمنة',
  'حالة',
  'مرضى',
  'الكلى',
  'الفيتامينات',
  'يمكن',
  'تضر',
  'مرضى',
  'الكلى',
  'أيضًا',
  'لذلك',
  'يجب',
  'استشارة',
  'الطبيب',
  'تناول',
  'نوع',
  'وتعتمد',
  'الخطة',
  'الغذائية',
  'لمريض',
  'الكلى',
  'حالة',
  'المريض',
  'مرحلة',
  'متقدمة',
  'متأخرة',
  'المرض',
  'وعليها',
  'يُوصَى',
  'بالحد',
  'تجنُّب',
  'الأطعمة',
  'الغذائية',
  'ويوصى',
  'لمريض',
  'الكلى',
  'بالآتي',
  'الحد',
  'الصوديوم',
  'مُضِي',
  'الوقت',
  'يفقد',
  'مريض',
  'الكلى',
  'تدريجيًّا',
  'القدرة',
  'تحقيق',
  'التوازن',
  'المياه',
  'والصوديوم',
  'بالجسم',
  'لذلك',
  'يساعد',
  'الحد',
  'الصوديوم',
  'خفض',
  'ضغط',
  'الدم',
  'وتقليل',
  'احتباس',
  'السوائل',
  'بالجسم',
  'أمر',
  'شائع',
  'مرضى',
  'الكلى',
  'التركيز',
  'الطعام',
  'الطازج',
  'والمطهو',
  'بالمنزل',
  'وت

In [139]:
import nltk
from nltk.stem import SnowballStemmer

# Download the SnowballStemmer resource for Arabic
nltk.download('snowball_data')

# Function to apply stemming to the words in the data
def apply_arabic_stemming(data):
    stemmer = SnowballStemmer('arabic')
    stemmed_data = []
    for entry in data:
        if 'words' in entry and entry['words']:
            # Apply stemming to each word in the list
            stemmed_words = [stemmer.stem(word) for word in entry['words']]
            # Update the 'words' key with the stemmed words
            entry['words'] = stemmed_words
            # Append the updated entry to the stemmed data list
            stemmed_data.append(entry)
    return stemmed_data

# Apply Arabic stemming to the tokenized data
stemmed_data = apply_arabic_stemming(tokenized_data)
stemmed_data[0]

[nltk_data] Downloading package snowball_data to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package snowball_data is already up-to-date!


{'illness_type': 'داء السكري',
 'disease_name': 'التغذية الصحية لمرضى السكري المصابين بأمراض الكلى',
 'words': ['الخط',
  'غذاء',
  'مرضى',
  'كلى',
  'مكمل',
  'عشب',
  'ليس',
  'امن',
  'في',
  'حال',
  'مرضى',
  'كلى',
  'كما',
  'أن',
  'بعض',
  'فيتامين',
  'يمك',
  'أن',
  'تضر',
  'مرضى',
  'كلى',
  'ايض',
  'لذل',
  'يجب',
  'استشار',
  'طبيب',
  'قبل',
  'تناول',
  'أي',
  'نوع',
  'منه',
  'تعتمد',
  'الخط',
  'غذاء',
  'مريض',
  'كلى',
  'على',
  'حال',
  'مريض',
  'وما',
  'اذا',
  'كان',
  'في',
  'مرحل',
  'متقدم',
  'أو',
  'متاخر',
  'من',
  'مرض',
  'وعل',
  'يوصى',
  'الحد',
  'من',
  'أو',
  'تجنب',
  'بعض',
  'اطعم',
  'غذاء',
  'يوصى',
  'مريض',
  'كلى',
  'الات',
  'الحد',
  'من',
  'صوديوم',
  'حيث',
  'مع',
  'مض',
  'الو',
  'يفقد',
  'مريض',
  'كلى',
  'تدريج',
  'قدر',
  'على',
  'تحقيق',
  'تواز',
  'بين',
  'مياه',
  'صوديوم',
  'جسم',
  'لذل',
  'يساعد',
  'الحد',
  'من',
  'صوديوم',
  'على',
  'خفض',
  'ضغط',
  'الدم',
  'تقليل',
  'احتباس',
  'سوايل',
  

In [146]:
import re
import nltk
from nltk.stem import WordNetLemmatizer

# Download necessary resources for Arabic lemmatization
nltk.download('wordnet')

def lemmatize_arabic(data):
    lemmatizer = WordNetLemmatizer()
    
    for entry in data:
        if 'words' in entry and entry['words']:
            # Lemmatize Arabic words
            words = entry['words']
            lemmatized_words = [lemmatizer.lemmatize(word, pos='v') for word in words]  # Assuming verbs for Arabic
            
            # Update entry with lemmatized words
            entry['lemmatized_words'] = lemmatized_words
        else:
            print("Skipping entry with missing or empty words:", entry)

    return data

# Example usage
lemmatized_data = lemmatize_arabic(stopwords_data)
lemmatized_data[0]

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


{'illness_type': 'داء السكري',
 'disease_name': 'التغذية الصحية لمرضى السكري المصابين بأمراض الكلى',
 'words': ['الخطة',
  'الغذائية',
  'لمرضى',
  'الكلى',
  'المكملات',
  'العشبية',
  'آمنة',
  'حالة',
  'مرضى',
  'الكلى',
  'الفيتامينات',
  'يمكن',
  'تضر',
  'مرضى',
  'الكلى',
  'أيضًا',
  'لذلك',
  'يجب',
  'استشارة',
  'الطبيب',
  'تناول',
  'نوع',
  'وتعتمد',
  'الخطة',
  'الغذائية',
  'لمريض',
  'الكلى',
  'حالة',
  'المريض',
  'مرحلة',
  'متقدمة',
  'متأخرة',
  'المرض',
  'وعليها',
  'يُوصَى',
  'بالحد',
  'تجنُّب',
  'الأطعمة',
  'الغذائية',
  'ويوصى',
  'لمريض',
  'الكلى',
  'بالآتي',
  'الحد',
  'الصوديوم',
  'مُضِي',
  'الوقت',
  'يفقد',
  'مريض',
  'الكلى',
  'تدريجيًّا',
  'القدرة',
  'تحقيق',
  'التوازن',
  'المياه',
  'والصوديوم',
  'بالجسم',
  'لذلك',
  'يساعد',
  'الحد',
  'الصوديوم',
  'خفض',
  'ضغط',
  'الدم',
  'وتقليل',
  'احتباس',
  'السوائل',
  'بالجسم',
  'أمر',
  'شائع',
  'مرضى',
  'الكلى',
  'التركيز',
  'الطعام',
  'الطازج',
  'والمطهو',
  'بالمنزل',
  'وت

In [149]:
!pip install farasa
!source venv/bin/activate

'source' n’est pas reconnu en tant que commande interne
ou externe, un programme exécutable ou un fichier de commandes.


In [147]:
from farasa.pos import FarasaPOSTagger
from farasa.lemma import FarasaLemmatizer

# Create Farasa POS Tagger and Lemmatizer objects
farasa_pos = FarasaPOSTagger()
farasa_lemmatizer = FarasaLemmatizer()

# Function to apply lemmatization to the words in the data
def apply_arabic_lemmatization(data):
    lemmatized_data = []
    for entry in data:
        if 'words' in entry and entry['words']:
            # Tag parts of speech for each word
            pos_tags = farasa_pos.tag(entry['words'])
            # Lemmatize each word based on its POS tag
            lemmatized_words = [farasa_lemmatizer.lemmatize(word, pos) for word, pos in pos_tags]
            # Update the 'words' key with the lemmatized words
            entry['words'] = lemmatized_words
            # Append the updated entry to the lemmatized data list
            lemmatized_data.append(entry)
    return lemmatized_data

# Apply Arabic lemmatization to the tokenized data
lemmatized_data = apply_arabic_lemmatization(tokenized_data)

ModuleNotFoundError: No module named 'farasa.pos'

In [ ]:
# Removing diacritics
def remove_diacritics(word_or_list):
    # Define a regular expression pattern to match Arabic diacritics
    pattern = r'[\u064B-\u065F\u0670]'  # Range of Arabic diacritical marks
    
    # If input is a list, apply remove_diacritics recursively to each element
    if isinstance(word_or_list, list):
        return [remove_diacritics(word) for word in word_or_list]
    
    # If input is a string, remove diacritics using the pattern
    elif isinstance(word_or_list, str):
        return re.sub(pattern, '', word_or_list)
    
    # Otherwise, return the input unchanged
    else:
        return word_or_list

# Apply remove_diacritics function to filtered_words
cleaned_words = remove_diacritics(filtered_words)

# Print words without diacritics
for i, words in enumerate(cleaned_words[:100]):  # Print the first few phrases
    print(f"Phrase {i+1} words (without diacritics):", words)

In [ ]:
# Normalization
import unicodedata

def normalize_words(words):
    normalized_words = []
    for word in words:
        # Check if the word is a list (nested list)
        if isinstance(word, list):
            # If it's a nested list, recursively call normalize_words
            normalized_sublist = normalize_words(word)
            normalized_words.append(normalized_sublist)
        else:
            # Unicode normalization (NFKD form)
            normalized_word = unicodedata.normalize('NFKD', word)
        
            # Case normalization (convert to lowercase)
            normalized_word = normalized_word.lower()
        
            # Whitespace normalization (replace multiple spaces with a single space)
            normalized_word = ' '.join(normalized_word.split())
        
            normalized_words.append(normalized_word)
    
    return normalized_words

# Normalize cleaned words
normalized_words = normalize_words(cleaned_words)

# Print normalized words
for i, word in enumerate(normalized_words[:100]):  # Print the first few normalized words
    print(f"Normalized word {i+1}:", word)

In [150]:
!pip install stanza


  Obtaining dependency information for stanza from https://files.pythonhosted.org/packages/03/3a/4554b9cd2218ff7d652406f27e957300b8fbae1700027367be2126196140/stanza-1.8.1-py3-none-any.whl.metadata
  Obtaining dependency information for emoji from https://files.pythonhosted.org/packages/55/c1/1d97d960831e23a3dd8608b0b489eb4866144445475277a443efb0379609/emoji-2.11.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for protobuf>=3.15.0 from https://files.pythonhosted.org/packages/8d/83/d70cb6fedb1f38318af01f0035f2103732630af0ca323c0198122b49323b/protobuf-5.26.1-cp310-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/970.4 kB ? eta -:--:--
   -- ------------------------------------- 61.4/970.4 kB 3.2 MB/s eta 0:00:01
   ------------------- -------------------- 481.3/970.4 kB 4.3 MB/s eta 0:00:01
   ------------------------ --------------- 604.2/970.4 kB 3.8 MB/s eta 0:00:01
   ---------------------------------------  962.6/970.4 kB 4.3 MB/s eta 0:0

In [152]:
import stanza